<a href="https://colab.research.google.com/github/NicoPozio/WildfireDetectionDL/blob/main/notebook/Classifier_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/NicoPozio/WildfireDetectionDL.git

Cloning into 'WildfireDetectionDL'...
remote: Enumerating objects: 318, done.
remote: Counting objects: 100% (130/130), done.
remote: Compressing objects: 100% (129/129), done.
remote: Total 318 (delta 70), reused 2 (delta 0), pack-reused 188 (from 1)
Receiving objects: 100% (318/318), 317.30 KiB | 10.24 MiB/s, done.
Resolving deltas: 100% (165/165), done.


In [2]:
import sys

repo_path = '/content/WildfireDetectionDL'
if repo_path not in sys.path:
    sys.path.append(repo_path)

In [3]:
# Cell: Update Code
import os

REPO_PATH = "/content/WildfireDetectionDL"

print("Syncing with GitHub...")
!cd {REPO_PATH} && git pull

print("Code updated. You can now run the Sweep or Train cell immediately.")

Syncing with GitHub...
Already up to date.
Code updated. You can now run the Sweep or Train cell immediately.


In [4]:
#Install Project Dependencies
#hydra-core: Configuration management
#wandb: Experiment tracking
#omegaconf: Dict handling for Hydra
!pip install -q hydra-core wandb omegaconf


from google.colab import drive
import os

if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')
    print("Google Drive Mounted")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 12.2 MB/s eta 0:00:00
Mounted at /content/drive
Google Drive Mounted


In [5]:
import wandb
from google.colab import userdata
import os

try:
    #Fetch key from Colab Secrets
    api_key = userdata.get('WANDB_API_KEY')

    #Set as Environment Variable
    #This ensures Hydra and subprocesses can find it automatically
    os.environ["WANDB_API_KEY"] = api_key

    wandb.login()
    print("Logged in to WandB")

except Exception as e:
    print(f"Authentication Failed: {e}")
    print("Action Required: Go to the 'Secrets' tab (Key icon) on the left.")
    print("Add a new secret named 'WANDB_API_KEY' with your key from https://wandb.ai/authorize")

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: Currently logged in as: pozioniccolo (pozioniccolo-sapienza-universit-di-roma) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Logged in to WandB


First we download the dataset, we check if the user has the data in its google drive, if not it's downloaded from a public link

In [6]:
import os
import gdown
from src.utils import extract_zip

#Destination on Colab
local_root = "/content/data"
os.makedirs(local_root, exist_ok=True)
!rm -rf {local_root}/*

#Paths on drive (in case the CyclaGAN notebook is exectued)
drive_dataset_path = "/content/drive/MyDrive/Wildfire_Project/dataset.zip"
drive_synthetic_path = "/content/drive/MyDrive/Wildfire_Project/synthetic_wildfire_2k.zip"

#Paths on Colab
target_dataset_zip = os.path.join(local_root, "dataset.zip")
target_synthetic_zip = os.path.join(local_root, "synthetic_wildfire_2k.zip")



#Check if the file are present
if os.path.isfile(drive_dataset_path) and os.path.isfile(drive_synthetic_path):
    !cp "{drive_dataset_path}" "{target_dataset_zip}"
    !cp "{drive_synthetic_path}" "{target_synthetic_zip}"

else:

    DATASET_ID = "17KPBVodZkmBYqz7252mDk6hfY55eMU-Q"
    SYNTHETIC_ID = "1KyI09FDCAkLp1BYO-VgD7zRMrtrl3anK"

    gdown.download(id=DATASET_ID, output=target_dataset_zip, quiet=False)

    gdown.download(id=SYNTHETIC_ID, output=target_synthetic_zip, quiet=False)


if os.path.exists(target_dataset_zip) and os.path.exists(target_synthetic_zip):
    if os.path.getsize(target_dataset_zip) > 0:
        extract_zip(target_dataset_zip, local_root)
    if os.path.getsize(target_synthetic_zip) > 0:
        extract_zip(target_synthetic_zip, local_root)
    print("Data Ready")

Extracting dataset.zip to /content/data...


Unzipping: 100%|██████████| 42860/42860 [00:13<00:00, 3225.75files/s]


 -> Success: Extracted to /content/data
Extracting synthetic_wildfire_2k.zip to /content/data...


Unzipping: 100%|██████████| 2003/2003 [00:01<00:00, 1327.73files/s]

 -> Success: Extracted to /content/data
Data Ready


In [7]:
import os
from PIL import Image
from tqdm import tqdm

# Define your data root
DATA_ROOT = "/content/data"

print(f"Scanning {DATA_ROOT} for corrupt images...")

corrupt_count = 0
for root, dirs, files in os.walk(DATA_ROOT):
    for filename in files:
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            file_path = os.path.join(root, filename)
            try:
                # Try to fully load the image bytes
                with Image.open(file_path) as img:
                    img.load()
            except OSError:
                print(f"Found corrupt image: {file_path}")
                try:
                    os.remove(file_path)
                    print(f"Deleted {filename}")
                    corrupt_count += 1
                except:
                    print(f"Could not delete {filename}")

print(f"\nScan Complete. Removed {corrupt_count} corrupt files.")

Scanning /content/data for corrupt images...
Found corrupt image: /content/data/dataset/test/wildfire/-73.15884,46.38819.jpg
Deleted -73.15884,46.38819.jpg
Found corrupt image: /content/data/dataset/train/nowildfire/-114.152378,51.027198.jpg
Deleted -114.152378,51.027198.jpg

Scan Complete. Removed 2 corrupt files.


In [ ]:
import subprocess
import os

# 1. Define Paths Constants
REPO_ROOT = "/content/WildfireDetectionDL"
SWEEP_CONFIG_PATH = os.path.join(REPO_ROOT, "conf", "sweep.yaml")

# 2. Validation
if not os.path.exists(SWEEP_CONFIG_PATH):
    print(f"CRITICAL ERROR: Could not find sweep.yaml at {SWEEP_CONFIG_PATH}")
else:
    print(f"Registering Sweep from {SWEEP_CONFIG_PATH}")

    # 3. Register the sweep
    result = subprocess.run(
        ["wandb", "sweep", SWEEP_CONFIG_PATH],
        capture_output=True,
        text=True,
        # CORRECTION: cwd must be the directory, not the file
        cwd=REPO_ROOT
    )

    output_text = result.stderr + result.stdout
    print("Raw Output:", output_text)

    # 4. Extract Sweep ID
    sweep_id = None
    for line in output_text.split('\n'):
        if "wandb agent" in line:
            parts = line.strip().split("wandb agent ")
            if len(parts) > 1:
                sweep_id = parts[-1].strip()
                break

    # 5. Launch the Agent
    if sweep_id:
        print(f"\nSUCCESS: Detected Sweep ID: {sweep_id}")
        print("Starting Agent... (This will run multiple experiments)")

        # CORRECTION: We use the explicit REPO_ROOT variable defined at the top
        !cd {REPO_ROOT} && wandb agent {sweep_id} --count 20
    else:
        print("\nERROR: Could not find Sweep ID.")

Registering Sweep from /content/WildfireDetectionDL/conf/sweep.yaml


KeyboardInterrupt: 

With the sweep execution we found the best configuration for the neural networks, now we train the resnet50 using that parameters, we train using both synthetic+real, and only real

In [8]:
import subprocess
import sys
import os


# ResNet50
RESNET_CONFIG = [
    "model.name=resnet50",
    "model.dropout=0.5",
    "dataset.augmentation.rotation_degrees=11",
    "training.batch_size=64",
    "training.learning_rate=0.00005836874490583941",
    "training.optimizer=adam",
    "training.weight_decay=0.00000621085260935666",
    "training.epochs=20",
    "dataset.params.real_data_fraction=0.1"
]

# SimpleCNN
SIMPLE_CNN_CONFIG = [
    "model.name=simple_cnn",
    "model.dropout=0.5",
    "dataset.augmentation.rotation_degrees=4",
    "training.batch_size=64",
    "training.learning_rate=0.00001349",
    "training.optimizer=adam",
    "training.weight_decay=0.00000823",
    "training.epochs=20",
    "dataset.params.real_data_fraction=0.1"
]

# EfficientNet
EFFICIENTNET_CONFIG = [
    "model.name=efficientnet",
    "model.dropout=0.2",
    "dataset.augmentation.rotation_degrees=15",
    "training.batch_size=32",
    "training.learning_rate=0.0001",
    "training.optimizer=adam",
    "training.weight_decay=0.0",
    "training.epochs=20",
    "dataset.params.real_data_fraction=0.1"
]


def run_training_pipeline(model_label, use_synthetic, specific_config):
    # Create a clear name for WandB
    run_name = f"{model_label}_10pct_{'Synthetic' if use_synthetic else 'RealOnly'}"

    print(f"STARTING: {run_name}")

    # Build the command
    cmd = [
        sys.executable, "train.py",
        "wandb.project=wildfire-final-benchmark",
        f"dataset.params.use_synthetic={str(use_synthetic).lower()}",
        f"+wandb.name={run_name}",      # Pass the name to Hydra/WandB
        f"+wandb.group={model_label}"   # Group by model architecture
    ] + specific_config

    try:
        # Use Popen to stream output
        process = subprocess.Popen(
            cmd,
            cwd="/content/WildfireDetectionDL",
            stdout=subprocess.PIPE,
            stderr=subprocess.STDOUT,
            text=True,
            bufsize=1,
            universal_newlines=True
        )

        # Stream output to console
        for line in process.stdout:
            print(line, end="")

        process.wait()

        if process.returncode != 0:
            print(f"\nFAILED: {run_name}")
        else:
            print(f"\nCOMPLETE: {run_name}")

    except Exception as e:
        print(f"CRITICAL ERROR: {e}")


#ResNet50
run_training_pipeline("ResNet50", False, RESNET_CONFIG) # Baseline (10% Real)
run_training_pipeline("ResNet50", True, RESNET_CONFIG)  # Experiment (10% Real + Synthetic)

#SimpleCNN
run_training_pipeline("SimpleCNN", False, SIMPLE_CNN_CONFIG)
run_training_pipeline("SimpleCNN", True, SIMPLE_CNN_CONFIG)

#EfficientNet
run_training_pipeline("EfficientNet", False, EFFICIENTNET_CONFIG)
run_training_pipeline("EfficientNet", True, EFFICIENTNET_CONFIG)

STARTING: ResNet50_10pct_RealOnly
wandb: Currently logged in as: pozioniccolo (pozioniccolo-sapienza-universit-di-roma) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: setting up run mp9dykcg
wandb: Tracking run with wandb version 0.23.0
wandb: Run data is saved locally in /content/WildfireDetectionDL/wandb/run-20251205_090235-mp9dykcg
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run ResNet50_10pct_RealOnly
wandb: ⭐️ View project at https://wandb.ai/pozioniccolo-sapienza-universit-di-roma/wildfire-final-benchmark
wandb: 🚀 View run at https://wandb.ai/pozioniccolo-sapienza-universit-di-roma/wildfire-final-benchmark/runs/mp9dykcg
DEBUG: Starting main function...
DEBUG: Initializing WandB...
DEBUG: WandB Initialized.
Training on cuda using model: resnet50
DEBUG: Loading Data...
Class Mapping Locked: {'nowildfire': 0, 'wildfire': 1}
   -> Synthetic data will use Class Index: 1
SCARCITY MODE: Using 10.0% of Real Data.
   Real Training Samples: 

In [9]:
import wandb
import subprocess
import sys
import os
import re
import pandas as pd

# Basic settings for the project
# Make sure you are logged in with wandb.login() before running this
PROJECT = "wildfire-final-benchmark"
ENTITY = wandb.Api().default_entity

# This is where the repository is cloned in Colab
REPO_DIR = "/content/WildfireDetectionDL"

# These are the exact configurations we want to benchmark
TARGETS = [
    ("ResNet50", False), ("ResNet50", True),
    ("SimpleCNN", False), ("SimpleCNN", True),
    ("EfficientNet", False), ("EfficientNet", True),
]

def parse_metrics(output_text):
    # We use regex here to pull the numbers out of the console output
    # This expects the test.py script to print lines like "Accuracy: 0.95"
    metrics = {}
    patterns = {
        "Accuracy": r"Accuracy:\s+([0-9.]+)",
        "Precision": r"Precision:\s+([0-9.]+)",
        "Recall": r"Recall:\s+([0-9.]+)",
        "F1-Score": r"F1-Score:\s+([0-9.]+)"
    }

    for key, pattern in patterns.items():
        match = re.search(pattern, output_text)
        if match:
            metrics[key] = float(match.group(1))
        else:
            # If we can't find the metric, default to 0 so the script doesn't crash
            metrics[key] = 0.0
    return metrics

def run_colab_testing_pipeline():
    api = wandb.Api()
    results_data = []

    print(f"Starting Benchmark Evaluation on Project: {PROJECT}\n")

    for model_label, use_synthetic in TARGETS:
        # Recreate the run name so we can find it in the cloud.
        # This matches the +wandb.name argument we used during training.
        run_name = f"{model_label}_10pct_{'Synthetic' if use_synthetic else 'RealOnly'}"
        print(f"PROCESSING: {run_name}")

        # Search for the run in WandB history
        runs = api.runs(f"{ENTITY}/{PROJECT}", filters={"display_name": run_name, "state": "finished"}, order="-created_at")
        if len(runs) == 0:
            print(f"[ERROR] Could not find a finished run named '{run_name}'. Skipping.")
            continue

        # If there are duplicates, the first one is usually the most recent
        target_run = runs[0]
        print(f" >> Found Run ID: {target_run.id}")

        # Check if this run actually has a model saved
        try:
            logged_artifacts = target_run.logged_artifacts()
            model_artifacts = [a for a in logged_artifacts if a.type == 'model']

            if not model_artifacts:
                print(f" >> [ERROR] Run exists but has no model artifact. Skipping.")
                continue

            artifact = model_artifacts[0]
            print(f" >> Downloading Artifact: {artifact.name}")

            # Download the weights to the local Colab disk
            download_dir = artifact.download()

            # We need the absolute path because the subprocess runs in a different folder
            weight_path = os.path.abspath(os.path.join(download_dir, "model_weights.pth"))

        except Exception as e:
            print(f" >> [ERROR] Problem downloading artifact: {e}")
            continue

        # Now we figure out the architecture details from the config
        # This is important so we don't accidentally load a ResNet into a SimpleCNN
        train_config = target_run.config
        try:
            # Sometimes config is nested (model -> dropout), sometimes flat (model.dropout)
            # We try both ways to be safe
            if 'model' in train_config and isinstance(train_config['model'], dict):
                model_dropout = train_config['model'].get('dropout', 0.0)
                model_arch_name = train_config['model'].get('name')
            else:
                model_dropout = train_config.get('model.dropout', 0.0)
                model_arch_name = train_config.get('model.name')

            # If the config is missing the name, assume it matches our loop label
            if not model_arch_name:
                model_arch_name = model_label.lower()

        except Exception:
            # If everything fails, use safe defaults
            print(" >> [WARNING] Could not read config. Using defaults.")
            model_dropout = 0.5
            model_arch_name = model_label.lower()

        print(f" >> Configuration inferred: Arch={model_arch_name}, Dropout={model_dropout}")

        # Prepare the command to run test.py
        cmd = [
            sys.executable, "test.py",
            f"model.name={model_arch_name}",
            f"model.dropout={model_dropout}",
            f"dataset.params.use_synthetic={str(use_synthetic).lower()}",
            "wandb.mode=disabled",
            f"+model_path={weight_path}"
        ]

        print(" >> Starting test execution...")

        # Using Popen allows us to see the output line by line as it happens
        try:
            process = subprocess.Popen(
                cmd,
                cwd=REPO_DIR,    # Run inside the repo folder so imports work
                stdout=subprocess.PIPE,
                stderr=subprocess.STDOUT, # Merge errors into main output
                text=True,
                bufsize=1,
                universal_newlines=True
            )

            # We need to capture the full output to parse metrics later,
            # but we also want to print it now so the user sees progress.
            full_console_output = ""

            for line in process.stdout:
                print(line, end="")
                full_console_output += line

            process.wait()

            if process.returncode == 0:
                print("\n >> [SUCCESS] Test finished.")

                # Extract the numbers from the text we just captured
                metrics = parse_metrics(full_console_output)

                entry = {
                    "Model": model_label,
                    "Data": "Synthetic" if use_synthetic else "RealOnly",
                    "Run_ID": target_run.id,
                    **metrics
                }
                results_data.append(entry)
            else:
                print("\n >> [FAILURE] The test script crashed.")

        except Exception as e:
            print(f" >> [ERROR] subprocess failed: {e}")

    # Finally, save everything to a CSV file
    if results_data:
        df = pd.DataFrame(results_data)
        print(f"\n{'='*60}")
        print("FINAL BENCHMARK RESULTS")
        print(f"{'='*60}")
        print(df)
        df.to_csv("benchmark_results.csv", index=False)
        print("\nResults saved to benchmark_results.csv")
    else:
        print("\nNo results gathered.")

# Run the function
if __name__ == "__main__":
    run_colab_testing_pipeline()

Starting Benchmark Evaluation on Project: wildfire-final-benchmark

PROCESSING: ResNet50_10pct_RealOnly
 >> Found Run ID: mp9dykcg
 >> Downloading Artifact: model-resnet50-mp9dykcg:v0


wandb: Downloading large artifact 'model-resnet50-mp9dykcg:v0', 90.00MB. 1 files...
wandb:   1 of 1 files downloaded.  
Done. 00:00:00.6 (152.6MB/s)


 >> Configuration inferred: Arch=resnet50, Dropout=0.5
 >> Starting test execution...
Testing Model: resnet50
DEBUG: Processing 320 keys from artifact...
>> SUCCESS: Weights loaded with strict=True
Class Mapping Locked: {'nowildfire': 0, 'wildfire': 1}
   -> Synthetic data will use Class Index: 1
   Real Training Samples: 30249
   Baseline Mode: Real Data Only

RESULTS:
Accuracy:  0.9490
Precision: 0.9713
Recall:    0.9353
F1-Score:  0.9530

 >> [SUCCESS] Test finished.
PROCESSING: ResNet50_10pct_Synthetic
 >> Found Run ID: e3jg24rr
 >> Downloading Artifact: model-resnet50-e3jg24rr:v0


wandb: Downloading large artifact 'model-resnet50-e3jg24rr:v0', 90.00MB. 1 files...
wandb:   1 of 1 files downloaded.  
Done. 00:00:00.4 (239.8MB/s)


 >> Configuration inferred: Arch=resnet50, Dropout=0.5
 >> Starting test execution...
Testing Model: resnet50
DEBUG: Processing 320 keys from artifact...
>> SUCCESS: Weights loaded with strict=True
Class Mapping Locked: {'nowildfire': 0, 'wildfire': 1}
   -> Synthetic data will use Class Index: 1
   Real Training Samples: 30249
SYNTHETIC INJECTION: Enabled
   -> Found 2000 synthetic wildfire images (Label: 1).
Total Training Size: 32249 (Real + Synthetic)

RESULTS:
Accuracy:  0.9600
Precision: 0.9521
Recall:    0.9767
F1-Score:  0.9642

 >> [SUCCESS] Test finished.
PROCESSING: SimpleCNN_10pct_RealOnly
 >> Found Run ID: wsl9bp4f
 >> Downloading Artifact: model-simple_cnn-wsl9bp4f:v0


...

 >> Configuration inferred: Arch=simple_cnn, Dropout=0.5
 >> Starting test execution...
Testing Model: simple_cnn
DEBUG: Processing 10 keys from artifact...
>> SUCCESS: Weights loaded with strict=True
Class Mapping Locked: {'nowildfire': 0, 'wildfire': 1}
   -> Synthetic data will use Class Index: 1
   Real Training Samples: 30249
SYNTHETIC INJECTION: Enabled
   -> Found 2000 synthetic wildfire images (Label: 1).
Total Training Size: 32249 (Real + Synthetic)

RESULTS:
Accuracy:  0.9127
Precision: 0.8867
Recall:    0.9652
F1-Score:  0.9243

 >> [SUCCESS] Test finished.
PROCESSING: EfficientNet_10pct_RealOnly
 >> Found Run ID: rotzgv38
 >> Downloading Artifact: model-efficientnet-rotzgv38:v0


wandb:   1 of 1 files downloaded.  


 >> Configuration inferred: Arch=efficientnet, Dropout=0.2
 >> Starting test execution...
Testing Model: efficientnet
DEBUG: Processing 360 keys from artifact...
>> SUCCESS: Weights loaded with strict=True
Class Mapping Locked: {'nowildfire': 0, 'wildfire': 1}
   -> Synthetic data will use Class Index: 1
   Real Training Samples: 30249
   Baseline Mode: Real Data Only

RESULTS:
Accuracy:  0.9727
Precision: 0.9750
Recall:    0.9756
F1-Score:  0.9753

 >> [SUCCESS] Test finished.
PROCESSING: EfficientNet_10pct_Synthetic
 >> Found Run ID: 8zc5d5k7
 >> Downloading Artifact: model-efficientnet-8zc5d5k7:v0


wandb:   1 of 1 files downloaded.  


 >> Configuration inferred: Arch=efficientnet, Dropout=0.2
 >> Starting test execution...
Testing Model: efficientnet
DEBUG: Processing 360 keys from artifact...
>> SUCCESS: Weights loaded with strict=True
Class Mapping Locked: {'nowildfire': 0, 'wildfire': 1}
   -> Synthetic data will use Class Index: 1
   Real Training Samples: 30249
SYNTHETIC INJECTION: Enabled
   -> Found 2000 synthetic wildfire images (Label: 1).
Total Training Size: 32249 (Real + Synthetic)

RESULTS:
Accuracy:  0.9748
Precision: 0.9681
Recall:    0.9868
F1-Score:  0.9774

 >> [SUCCESS] Test finished.

FINAL BENCHMARK RESULTS
          Model       Data    Run_ID  Accuracy  Precision  Recall  F1-Score
0      ResNet50   RealOnly  mp9dykcg    0.9490     0.9713  0.9353    0.9530
1      ResNet50  Synthetic  e3jg24rr    0.9600     0.9521  0.9767    0.9642
2     SimpleCNN   RealOnly  wsl9bp4f    0.9086     0.9270  0.9057    0.9163
3     SimpleCNN  Synthetic  994zob9z    0.9127     0.8867  0.9652    0.9243
4  EfficientNet